#### План отчета:
&nbsp;

**1.	Постановка задачи и пути решения.**

&#8195;Зачем нужно? Почему важно? Как решать?


**2.	Первичный анализ данных**

&#8195;Что имеем? Стратегия работы

**3.	Предобработка выборки**

&#8195;Как преобразуем выборку что бы следовать стратегии работы?
>Можно выполнить сравнение (с применением виджетов) работы функции, считающей признаки для выборки. С добавлением значений по одному в соответствующую ячейку dataframe и добавлением по-столбцам (разница ~20 раз)

**4.	Проверка гипотез**

&#8195;Какие показатели значимо выделают одних пользователей от других? Почему?

**5.	Features development**

&#8195;Какие фичи будем пробовать? Почему? Что получилось?

**6.	Обучение и каггл**

&#8195;Почему бинарная классификация показывает себя хорошо, а многоклассовая дает плохие показатели метрик. Сравим метрики на каггле при разном количестве признаков

**7.	Многоклассовая классификация с Ворпал вэббит и SGDlogreg**

&#8195;Почему это в принципе так неэффективно и что менее неэффективно.

**8.	Вывод по решению задачи на данный момент, возможности ее выкатывания в продакшн, а так же дальнейшая стратегия по исследованию данной темы.**


### Содержание
&nbsp;
1. [Цель и задачи проекта. Описание исходных и обработанных данных и признаков](#1part)
2. [Подготовка данных и первичный анализ](#2part);
3. Разработка признаков и их визуальный анализ;
5. Подбор модели;
6. Итоги, выводы и описание дальнейшей работы

<a id='1part'></a>
### 1. Цель и задачи проекта. Описание исходных и обработанных данных и признаков
&nbsp;

1.1 Цель проекта

&#8195;Проект направлен на решение задачи идентификации пользователя по его поведению в сети Интернет. Задача как таковая является широкой в смысле выбора используемых признаков и стратегии. К примеру идентифицировать пользователя можно как по тому какие сайты в какое время он посещает, так и по тому как он это делает - какими разделами сайта он пользуется, как быстро отвечает на сообщения в социальных сетях, насколько длинны оставляемые им комментарии, или он предпочитает их не оставлять и т.д.

&#8195;Целью данного проекта является построение алгоритма идентификации пользователя по имеющимся данным с последовательностями посещенных пользователями сайтов и времени посещения каждого из них.

&nbsp;

1.2 Задачи проекта

- Предобработка данных, подготовка их к классификации
- Первичный и визуальный анализ
- Разработка признаков и их описание
- Сравнение семейств моделей классификации и подбор параметров выбранной модели
- Оценка метрик и вывод по проделанной работе
- Прогноз по дальнейшей работе


<a id='2part'></a>
### 2. Подготовка данных и первичный анализ
&nbsp;



Имеются данные с прокси-серверов Университета Блеза Паскаля.

Для каждого пользователя заведен csv-файл с названием user****.csv (где вместо звездочек – 4 цифры, соответствующие ID пользователя), а в нем посещения сайтов записаны в следующем формате: 

&nbsp;

<center>*timestamp, посещенный веб-сайт*</center>
&nbsp;

В дальнейшем будет использоваться параллельно 2 выборки: по 10 и по 150  пользователям. Для 10 пользователей будет отлаживаться код, для 150 будет использоваться рабочая версия.

Вид исходных данных приведен ниже. 

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">timestamp</th>
    <th class="tg-yw4l">site</th>
  </tr>
  <tr>
    <td class="tg-yw4l">2013-11-15 08:12:07</td>
    <td class="tg-yw4l">fpdownload2.macromedia.com</td>
  </tr>
  <tr>
    <td class="tg-yw4l">2013-11-15 08:12:17</td>
    <td class="tg-yw4l">www.laposte.net</td>
  </tr>
  <tr>
    <td class="tg-yw4l">2013-11-15 08:12:17</td>
    <td class="tg-yw4l">www.google.com</td>
  </tr>
  <tr>
    <td class="tg-yw4l">2013-11-15 08:12:18</td>
    <td class="tg-yw4l">www.laposte.net</td>
  </tr>
</table>

На данном этапе поставим задачу классификации: идентифицировать пользователя по сессии из 10 подряд посещенных сайтов. 

Объектом в этой задаче будет сессия из 10 сайтов, последовательно посещенных одним и тем же пользователем, признаками – индексы этих 10 сайтов. Целевым классом будет id пользователя.

Отображение сайтов в их индесы должно получиться таким:
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">site</th>
    <th class="tg-yw4l">site_id</th>
  </tr>
  <tr>
    <td class="tg-yw4l">vk.com</td>
    <td class="tg-yw4l">1</td>
  </tr>
  <tr>
    <td class="tg-yw4l">google.com</td>
    <td class="tg-yw4l">2</td>
  </tr>
  <tr>
    <td class="tg-yw4l">yandex.ru</td>
    <td class="tg-yw4l">3</td>
  </tr>
  <tr>
    <td class="tg-yw4l">facebook.com</td>
    <td class="tg-yw4l">4</td>
  </tr>
</table>

Тогда обучающая выборка для сессии длины 2 будет иметь вид (целевой признак – user_id):
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-s6z2{text-align:center}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-hgcj{font-weight:bold;text-align:center}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-hgcj">session_id</th>
    <th class="tg-hgcj">site1</th>
    <th class="tg-hgcj">site2</th>
    <th class="tg-amwm">user_id</th>
  </tr>
  <tr>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">2</td>
    <td class="tg-baqh">1</td>
  </tr>
  <tr>
    <td class="tg-s6z2">2</td>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-baqh">1</td>
  </tr>
  <tr>
    <td class="tg-s6z2">3</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-s6z2">2</td>
    <td class="tg-baqh">2</td>
  </tr>
  <tr>
    <td class="tg-s6z2">4</td>
    <td class="tg-s6z2">4</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-baqh">2</td>
  </tr>
</table>


Для подготовки обучающей выборки реализована функция  prepare_train_set, которая принимает на вход путь к каталогу с csv-файлами path_to_csv_files и параметр session_length – длину сессии, а возвращает 2 объекта: 
- DataFrame, в котором строки соответствуют уникальным сессиям из session_length сайтов, session_length столбцов – индексам этих session_length сайтов и последний столбец – ID пользователя
- частотный словарь сайтов вида {'site_string': [site_id, site_freq]}, например для недавнего игрушечного примера это будет {'vk.com': (1, 2), 'google.com': (2, 2), 'yandex.ru': (3, 3), 'facebook.com': (4, 1)}

    def func1(data, filename, session_length=10):
        """takes user history table and returns array with sessions history where number of 
        columns == session_length+1 (the last column is user's index)
        fills empty values in last session (if there is) with 0
        """

        i = True
        while i:
            try:
                data = data.reshape((-1, session_length))
                i = False
            except:
                data = np.append(data, (0))
        #     Add column with user index (index is pulled from filename):
        userindex = re.findall(r'(\d*).csv', filename)[0]
        userindexcolumn = np.array([userindex]*len(data))[:, np.newaxis]
        data = np.hstack((data, userindexcolumn))
        return data


    def func2(path_to_csv_files):
        '''Parce whole files' catalog. Applies func1 to all files.
        Returns Numpy array with sessions, dictionary with websites appearence frequency
        '''

        dic = {}
        data = None
        os.chdir(path_to_csv_files)
        filenames = sorted(os.listdir())

        for filename in tqdm_notebook(filenames):
            if '.csv' in filename:
                usersiteslist = pd.read_csv(os.path.join(path_to_csv_files, filename)).site.values
                counts = np.unique(usersiteslist, return_counts=True)

        #         Counting websites appearances:
                for i, j in zip(counts[0], counts[1]):
                    try:
                        dic[i] += j
                    except KeyError:
                        dic[i] = j    

        #         Filling the massive with sessions:
                try:
                    data = np.append(data, func1(usersiteslist, filename), axis=0) 
                except ValueError:
                    data = func1(usersiteslist, filename)
        return data, dic


    def func3(data, dic):
        '''
        Sorts websites' names by frequency and attach indeces in order where 1 is the most frequent 
        website. Replaces websites' names with theirs indeces.
        Returns: dict {'site_string': [site_id, site_freq]}; numpy array with user-connected sessions
        '''

        # Create list representation of dict and sort websites by appearing frequency:
        counts = [(key, value) for key, value in dic.items()]
        counts = sorted(counts, key = lambda x: x[1], reverse=True)

        # Create dict
        dic = {}
        i = 1
        for key, value in counts:
            dic[key] = [i, value]
            i += 1
        for i in range(data.shape[0]):
            for j in range(data[:, 0:-1].shape[1]):
                if data[i][j] != 0:
                    data[i][j] = dic[data[i][j]][0]
        return data, dic


    def prepare_train_set(path_to_csv_files, session_length=10):
        '''wrapper above func1, func2, func3.
        Takes path to .csv tables and session lenght (10 by default)
        Returns: DataFrame with user-connected indexed sessions; 
                 dict {'site_string': [site_id, site_freq]}
        '''

        data, dic = func2(path_to_csv_files)
        data, dic = func3(data, dic)
        columns = list(map(lambda x: 'site'+str(x), range(1, data.shape[1]))) + ['user_id']
        data = pd.DataFrame(data, columns=columns)
        data.index.name = 'session_id'
        return data, dic

>!После применения функции к данным для дальнейшей работы с полученными объектами DataFrame таблицы сохранены в csv-файлы:
    
>!    train_data_10users.to_csv(os.path.join(PATH_TO_DATA, 'train_data_10users.csv'), 
                              index_label='session_id', float_format='%d')
    train_data_150users.to_csv(os.path.join(PATH_TO_DATA, 'train_data_150users.csv'), 
                              index_label='session_id', float_format='%d')

VISIBLE CONTENT HERE
<div id="spoiler" style="display:none">
HIDDEN CONTENT HERE
</div>
<button title="Click to show/hide content" type="button" onclick="if(document.getElementById('spoiler') .style.display=='none') {document.getElementById('spoiler') .style.display=''}else{document.getElementById('spoiler') .style.display='none'}">Show/hide</button>